# Práctica Python
## Alumnos: _Tania Batista, Fernando de la Fuente, Lourdes Ruiz_
## Tema: Análisis del S&P 500
### Profesor: _Miguel Angel Sicilia_

Hemos elegido para la práctica una tabla de datos que contiene las cotizaciones de 503 valores que componen el índice de bolsa americana S&P 500 de 5 años, desde el 13 de Agosto de 2012 hasta el 11 de Agosto de 2017.
Los datos están extraídos de Kaggle, en el siguiente repositorio:

[Kaggle S&P 500](https://www.kaggle.com/camnugent/sandp500/)

A partir de estos datos, sin tener los datos del índice, sino los de los valores que lo componen, haremos algunos estudios, así como la composición de un índice sintético.
Deicidimos componer un sintético debido a que:
* El índice real varía diariamente el peso de cada uno de los valores que lo componen, y el acceso a esta información que es necesaria para componer el índice real es de pago. 
* Periodicamente se reune un comité que decide qué valores entran y salen del índice, y a lo largo de los 5 años de los que tenemos datos ha habido varios valores que han entrado y otros que han salido. Nuestra base de datos contiene información de los 503 valores que componían el índice en fecha 14 de Agosto de 2017, y sobre esta información es sobre la que trabajaremos.

Por tanto, debido a estas limitaciones no es posible recomponer el índice real y comprobar los datos obtenidos con los de cualquier otra fuente, y en consecuencia fabricaremos uno sintético asumiendo que en el periodo no ha variado la composición, y todos los valores tienen el mismo peso.

Comenzamos por tanto la práctica.

Lo primero sería cargar los datos, y volcarlos a un data frame.

In [44]:
import pandas as pd
#import csv, operator
path ='C:\\Users\\ffuen\\Desktop\\all_stocks_5yr.csv'
#path ='https://www.kaggle.com//camnugent//sandp500//downloads//all_stocks_5yr.csv'
sp500 = pd.read_csv(path, delimiter=',',decimal='.', header=0)
sp = pd.DataFrame(sp500)
print(sp.head(10))
print(sp.tail(10))

         Date   Open   High    Low  Close     Volume Name
0  2012-08-13  92.29  92.59  91.74  92.40  2075391.0  MMM
1  2012-08-14  92.36  92.50  92.01  92.30  1843476.0  MMM
2  2012-08-15  92.00  92.74  91.94  92.54  1983395.0  MMM
3  2012-08-16  92.75  93.87  92.21  93.74  3395145.0  MMM
4  2012-08-17  93.93  94.30  93.59  94.24  3069513.0  MMM
5  2012-08-20  94.00  94.17  93.55  93.89  1640008.0  MMM
6  2012-08-21  93.98  94.10  92.99  93.21  2302988.0  MMM
7  2012-08-22  92.56  93.36  92.43  92.68  2463908.0  MMM
8  2012-08-23  92.65  92.68  91.79  91.98  1823757.0  MMM
9  2012-08-24  92.03  92.97  91.94  92.83  1945796.0  MMM
              Date   Open   High    Low  Close     Volume Name
606791  2017-07-31    NaN    NaN  62.32  62.52  2130374.0  ZTS
606792  2017-08-01  62.60  62.69  61.92  62.27  1750974.0  ZTS
606793  2017-08-02  62.04  62.21  61.38  61.54  1831228.0  ZTS
606794  2017-08-03  61.23  62.47  61.20  62.28  1768815.0  ZTS
606795  2017-08-04  62.37  62.85  61.86  62.12 

Comprobamos la cantidad de información que tiene la tabla, en filas y columnas:

In [41]:
print(sp.index)
print(sp.columns)

RangeIndex(start=0, stop=606801, step=1)
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Name'], dtype='object')


Como hemos visto anteriormente, la tabla hacia el final tiene algunos campos vacíos, que se suplen con 'NaN'. 

Es necesario identificarlos, y sustituirlos para limpiar la tabla antes de trabajar con ella:

In [42]:
print("El número de campos vacíos en la columna Date es",sp["Date"].isnull().sum())
print("El número de campos vacíos en la columna Open es",sp["Open"].isnull().sum())
print("El número de campos vacíos en la columna High es",sp["High"].isnull().sum())
print("El número de campos vacíos en la columna Low es",sp["Low"].isnull().sum())
print("El número de campos vacíos en la columna Close es",sp["Close"].isnull().sum())
print("El número de campos vacíos en la columna Volume es",sp["Volume"].isnull().sum())
print("El número de campos vacíos en la columna Name es",sp["Name"].isnull().sum())


El número de campos vacíos en la columna Date es 0
El número de campos vacíos en la columna Open es 384
El número de campos vacíos en la columna High es 208
El número de campos vacíos en la columna Low es 227
El número de campos vacíos en la columna Close es 0
El número de campos vacíos en la columna Volume es 406
El número de campos vacíos en la columna Name es 0


Como vemos, los campos vacíos se encuentran en algunas columnas, no todas. 
Antes de decidir qué hacer con ellos, hagamos un análisis de qué información es la que contiene cada columna:
* _Date:_ Contiene la fecha de la información de cada fila.
* _Open:_ Es el primer precio al que abrió el valor ese día.
* _High:_ El precio más alto negociado en el día.
* _Low:_ El precio más bajo negociado en el día.
* _Close:_ Indica a qué precio cerró el valor ese día. Este podemos decir que es el dato más importante de la fila.
* _Volume:_ Indica el número de acciones negociadas en el día. No todas cambian de manos al mismo precio.
* _Name:_ Indica el nombre del valor. Es una abreviatura. Por ejemplo: Amazon sería AMZN.